In [4]:
import json
import faiss
import numpy as np
import pandas as pd
from google.cloud import storage
from google.cloud import aiplatform
from google.oauth2 import service_account
from sentence_transformers import SentenceTransformer
from vertexai.preview.generative_models import GenerativeModel

# Configurar o cliente do Google Cloud Storage
client = storage.Client()

# Seleção do Modelo
model =  GenerativeModel("gemini-1.5-pro-002")

# Listar buckets como exemplo de operação
buckets = list(client.list_buckets())

print("Buckets disponíveis no projeto:")
for bucket in buckets:
    print(bucket.name)

Buckets disponíveis no projeto:
estudo-ia-ds-datasets


# Prepara o banco de dados RAG

In [5]:
data = {
    "id": [1, 2],
    "text": ["O Banco BV usa o Catálogo de Dados Openmetadata", "O Banco Itaú usa o Catálogo de Dados Attlas"]
}

df = pd.DataFrame(data)

In [6]:
# Carregar o modelo pré-treinado
encoder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Gerar embeddings para a coluna 'text'
df['embedding'] = df['text'].apply(lambda x: encoder.encode(x).tolist())

In [8]:
dados = np.vstack(df['embedding'].values).astype('float32')

# Normalizar os vetores para busca por similaridade de cosseno
faiss.normalize_L2(dados)

# Criar o índice FAISS
index = faiss.IndexFlatIP(dados.shape[1])  # IP para Produto Interno (similaridade de cosseno)
index.add(dados)  # Adicionar dados ao índice

# Realiza a consulta

In [9]:
def retrieve_information(query, k=1):

    string_return = ''
    
    # Vetor de consulta
    consulta = encoder.encode(query).astype('float32').reshape(1, -1)
    faiss.normalize_L2(consulta)
    
    # Realizar a busca
    distancias, indices = index.search(consulta.reshape(1, -1), k)  # k é o número de vizinhos mais próximos

    # Exibir os textos correspondentes 
    for idx in indices[0]:
        string_return += df.iloc[idx]['text'] + ' \n'
    return string_return

def generate_query(query):
    retrieved_info = retrieve_information(query)
    prompt = f"{retrieved_info}\n\nBaseado na informação acima, responda: {query}"
    return prompt

In [10]:
response = model.generate_content(generate_query('Conte-me algo sobre o catálogo de dados do Banco BV'))

print(response.text)

A informação fornecida diz que o Banco BV *usa* o Catálogo de Dados OpenMetadata. Isso nos diz que eles escolheram essa plataforma específica para gerenciar seus metadados.  No entanto, não nos diz nada *específico* sobre *como* o BV usa o OpenMetadata.

Para saber mais sobre o catálogo de dados do BV especificamente, precisaríamos de informações adicionais.  Por exemplo:

* **Quais tipos de dados eles catalogam?** (Ex: dados transacionais, dados de clientes, dados de marketing, etc.)
* **Como eles integraram o OpenMetadata com seus sistemas existentes?**
* **Quais recursos do OpenMetadata eles utilizam mais?** (Ex: linhagem de dados, descoberta de dados, governança de dados, etc.)
* **Qual o tamanho e a complexidade do seu catálogo?** (Quantos datasets, tabelas, colunas, etc.)
* **Quais os benefícios que eles obtiveram com a implementação do OpenMetadata?** (Ex: melhor descoberta de dados, maior qualidade dos dados, melhor conformidade com regulamentações, etc.)

Sem essas informações